In [ ]:
## Data Ingestion
# # web based loader
# from langchain_community.document_loaders import WebBaseLoader
# import bs4

# ## load,chunk and index the content of the html page

# loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#                      bs_kwargs=dict(parse_only=bs4.SoupStrainer(
#                          class_=("post-title","post-content","post-header")

#                      )))

# text_documents=loader.load()
## Pdf reader
# from langchain_community.document_loaders import PyPDFLoader
# loader=PyPDFLoader('attention.pdf')
# docs=loader.load()
from langchain_community.document_loaders import TextLoader
loader=TextLoader("simple.txt")
text_documents=loader.load()

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(text_documents)

In [7]:
## Vector Embedding And Vector Store
from langchain_community.embeddings.ollama import (
        OllamaEmbeddings)
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents,embedding=OllamaEmbeddings(model='llava:latest'))

In [5]:
query = "Learn more about how we live our values at GitLab?"
retireved_results=db.similarity_search(query)
print(retireved_results)

NameError: name 'db' is not defined

In [8]:
from langchain_community.llms import Ollama
llm = Ollama(model="llava:latest")
llm

Ollama(model='llava:latest')

In [9]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context
Think step by step before providing a detailed answer
<context>
{context}
</context>
Question: {input}""")

In [11]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm,prompt)

In [12]:
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x137080e00>, search_kwargs={})

In [13]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [20]:
response = retrieval_chain.invoke({"input":"who is the 40th president of united states "})
response['answer']

" I'm sorry, but I cannot answer your question as it does not fit within the provided context. The information in the context seems to be related to job grades, hiring criteria, and company values for GitLab, rather than historical facts like the 40th president of the United States. If you have any questions about the context itself or if there is anything else I can assist you with, please let me know! "